In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
product_link = 'https://www.tokopedia.com/kiveeofficial/powerbank-20000mah-kivee-asli-masukan-dan-keluaran-ganda-5v2-1a-pink'

## Open Product Page

In [3]:
driver = webdriver.Chrome()
driver.get(product_link)

In [4]:
time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')

## Open Product's View Reviews Page

In [5]:
all_review = soup.find_all('a', attrs={'data-testid' : 'btnViewAllFeedback'})
print(all_review)

[<a class="all-review css-1wt74j6-unf-link e1u528jj0" data-testid="btnViewAllFeedback" data-unify="Link" font-size="14" href="/kiveeofficial/powerbank-20000mah-kivee-asli-masukan-dan-keluaran-ganda-5v2-1a-pink/review">Lihat Semua Ulasan</a>]


In [6]:
link_all_review = all_review[0]['href']
link_all_review = 'https://www.tokopedia.com' + link_all_review

In [7]:
driver.get(link_all_review)
time.sleep(5)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
reviews = []

## Get Num Of Pages

In [9]:
total_pages = soup.find_all('button', attrs={'class' : 'css-bugrro-unf-pagination-item'})
num_page = 0

for page in total_pages :
    if not page.text.isdigit() :
        continue

    if int(page.text) > num_page :
        num_page = int(page.text)

In [10]:
print(f'total pages : {num_page}')

total pages : 50


## Get the Reviews

In [11]:
toped_reviews = soup.find_all('article', attrs={'class' : 'css-72zbc4'})
print(len(toped_reviews))

50


In [12]:
for i in range (0, num_page-1):
    toped_reviews = soup.find_all('article', attrs={'class' : 'css-72zbc4'})
    for review in toped_reviews:
        # blank review checking
        if review.find('span', attrs={'data-testid' : 'lblItemUlasan'}) is not None : 
            text_review = review.find('span', attrs={'data-testid' : 'lblItemUlasan'}).text
            rating = review.find('div', attrs={'data-testid' : 'icnStarRating'})['aria-label']
            rating = rating.removeprefix('bintang ')
            reviews.append([text_review, rating])
    time.sleep(3)
    # click next page  
    try :
        driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Laman berikutnya"]').click()
    except :
        print('No more next page')
        break
    time.sleep(3)

No more next page


In [13]:
print(f'Jumlah review : {len(reviews)}')

Jumlah review : 423


In [14]:
for review in reviews:
    print(review)

['baru dipakai 2 udah rusak \n👎👎👎👎', '1']
['kirain mau pake dus gitu, eh ternyata engga...', '5']
['kualitas sesuai harga, packaging aman, semoga awet', '4']
['mantap, belum di coba', '5']
['berfungsi dengan baik', '5']
['👍👍👍👍', '5']
['bagus juga', '5']
['utk hp opo ku sepertinya hanya bisa 2 kali aj ngecas penuh', '4']
['Barang datang dalam kedaaan bagus walaupun tempatnya ada yang robek, belum dicoba semoga bisa dipakai awet', '4']
['barang bagus mantap, untuk ketahanan baterai belum di coba', '5']
['sumpah saya menyesal beli di toko ini barang rusak tidak bisa dipakai namun saldo saya tidak dikembalikan saya tidak ikhlas lahir dan batin akan saya tagih nanti di akhir... ', '1']
['barangnya sesuai deskripsi dan sangat memuaskan', '5']
['HATI HATI SEKALI UNTUK BELANJA DISINI.\nJANGAN BELANJA DISINI.\n\ndi Online memang terkesan banyak bintang 5, tetapi yang kasi bintang 5 itu adalah KORBAN RETUR, barang rusa... ', '1']
['mantapp', '5']
['barang original... kapasitas besar, harga segin

In [15]:
review_df = pd.DataFrame(reviews, columns=['review', 'rating'])
review_df.sort_values(by='rating', ascending=True, inplace=True)

In [16]:
review_df

,review,rating
0,baru dipakai 2 udah rusak \n👎👎👎👎,1
116,"Pertama buka dan pake, indikator 99%. Dipake g...",1
111,Barangnya tidak sampai namun pesanan telah sel...,1
339,sumpah saya menyesal beli di toko ini barang r...,1
106,HATI HATI SEKALI UNTUK BELANJA DISINI.\nJANGAN...,1
...,...,...
154,mantapp,5
152,barangnya sesuai deskripsi dan sangat memuaskan,5
150,"barang bagus mantap, untuk ketahanan baterai b...",5
162,semoga awet,5


In [17]:
# export to csv
review_df.to_csv('tokopedia_review.csv', index=False)